In [1]:
import numpy as np
import pandas as pd

In [2]:
# A MULTIPLY INDEXED SERIES
# --- The bad way ---
index = [('California', 2000), ('California', 2010),
         ('New York', 2000), ('New York', 2010),
         ('Texas', 2000), ('Texas', 2010)]
populations = [33871648, 37253956,
               18976457, 19378102,
               20851820, 25145561]
pop = pd.Series(populations, index=index)
pop

(California, 2000)    33871648
(California, 2010)    37253956
(New York, 2000)      18976457
(New York, 2010)      19378102
(Texas, 2000)         20851820
(Texas, 2010)         25145561
dtype: int64

In [3]:
 pop[('California', 2010):('Texas', 2000)]

(California, 2010)    37253956
(New York, 2000)      18976457
(New York, 2010)      19378102
(Texas, 2000)         20851820
dtype: int64

In [5]:
pop[[i for i in pop.index if i[1] == 2010]]

(California, 2010)    37253956
(New York, 2010)      19378102
(Texas, 2010)         25145561
dtype: int64

In [6]:
# --- The better way: Pandas MultiIndex ---
index = pd.MultiIndex.from_tuples(index)
index

MultiIndex([('California', 2000),
            ('California', 2010),
            (  'New York', 2000),
            (  'New York', 2010),
            (     'Texas', 2000),
            (     'Texas', 2010)],
           )

In [23]:
pop = pop.reindex(index)
pop

California  2000    33871648
            2010    37253956
New York    2000    18976457
            2010    19378102
Texas       2000    20851820
            2010    25145561
dtype: int64

In [8]:
# Now to access all data for which the second index is 2010, we can simply use the Pandas slicing notation:
pop[:, 2010]

California    37253956
New York      19378102
Texas         25145561
dtype: int64

In [9]:
# --- MultiIndex as extra dimension ---
pop_df = pop.unstack()
pop_df

,2000,2010
California,33871648,37253956
New York,18976457,19378102
Texas,20851820,25145561


In [10]:
pop_df.stack()

California  2000    33871648
            2010    37253956
New York    2000    18976457
            2010    19378102
Texas       2000    20851820
            2010    25145561
dtype: int64

In [11]:
# Concretely, we might want to add another column of demographic data for each state at each year (say, population under 18); with a MultiIndex this is as easy as adding another col‐
# umn to the DataFrame:
pop_df = pd.DataFrame({'total': pop,
                      'under18': [9267089, 9284094,
                                  4687374, 4318033,
                                  5906301, 6879014]})
pop_df

total  under18
California 2000  33871648  9267089
           2010  37253956  9284094
New York   2000  18976457  4687374
           2010  19378102  4318033
Texas      2000  20851820  5906301
           2010  25145561  6879014

In [13]:
f_u18 = pop_df['under18'] / pop_df['total']
f_u18.unstack()

,2000,2010
California,0.273594,0.249211
New York,0.247010,0.222831
Texas,0.283251,0.273568


In [15]:
# METHODS OF MULTIINDEX CREATIONs
# The most straightforward way to construct a multiply indexed Series or DataFrame is to simply pass a list of two or more index arrays to the constructor
df = pd.DataFrame(np.random.rand(4, 2),
                 index=[['a', 'a', 'b', 'b'], [1, 2, 1, 2]],
                 columns=['data1', 'data2'])
df

data1     data2
a 1  0.171789  0.409673
  2  0.106530  0.740357
b 1  0.117039  0.260040
  2  0.078316  0.221299

In [16]:
# Similarly, if you pass a dictionary with appropriate tuples as keys, Pandas will automatically recognize this and use a MultiIndex by default:
data =  {('California', 2000): 33871648,
         ('California', 2010): 37253956,
         ('Texas', 2000): 20851820,
         ('Texas', 2010): 25145561,
         ('New York', 2000): 18976457,
         ('New York', 2010): 19378102}
pd.Series(data)

California  2000    33871648
            2010    37253956
Texas       2000    20851820
            2010    25145561
New York    2000    18976457
            2010    19378102
dtype: int64

In [17]:
# --- Explicit MultiIndex Constructors ---
# you can construct the MultiIndex from a simple list of arrays, giving the index values within each level:
pd.MultiIndex.from_arrays([['a', 'a', 'b', 'b'], [1, 2, 1, 2]])

MultiIndex([('a', 1),
            ('a', 2),
            ('b', 1),
            ('b', 2)],
           )

In [18]:
# You can construct it from a list of tuples, giving the multiple index values of each point:
pd.MultiIndex.from_tuples([('a', 1), ('a', 2), ('b', 1), ('b', 2)])

MultiIndex([('a', 1),
            ('a', 2),
            ('b', 1),
            ('b', 2)],
           )

In [19]:
# You can even construct it from a Cartesian product of single indices:
pd.MultiIndex.from_product([['a', 'b'], [1, 2]])

MultiIndex([('a', 1),
            ('a', 2),
            ('b', 1),
            ('b', 2)],
           )

In [25]:
# --- MultiIndex level names ---
pop.index.names = ['state', 'year']
pop

state       year
California  2000    33871648
            2010    37253956
New York    2000    18976457
            2010    19378102
Texas       2000    20851820
            2010    25145561
dtype: int64

In [28]:
# --- MultiIndex for columns ---
# In a DataFrame, the rows and columns are completely symmetric, and just as the rows
# can have multiple levels of indices, the columns can have multiple levels as well

# hierarchical indices and columns
index = pd.MultiIndex.from_product([[2013, 2014], [1, 2]],
                                   names=['year', 'visit'])
columns = pd.MultiIndex.from_product([['Bob', 'Guido', 'Sue'], ['HR', 'Temp']],
                                     names=['subject', 'type'])

# mock some data
data = np.round(np.random.randn(4, 6), 1)
data[:, ::2] *= 10
data += 37

# create the DataFrame
health_data = pd.DataFrame(data, index=index, columns=columns)
health_data

subject      Bob       Guido         Sue      
type          HR  Temp    HR  Temp    HR  Temp
year visit                                    
2013 1      29.0  40.2  38.0  36.4  36.0  38.4
     2      31.0  36.9  36.0  36.6  20.0  37.3
2014 1      32.0  35.6  37.0  36.5  44.0  36.3
     2      50.0  37.2  37.0  38.5  37.0  37.5

In [29]:
health_data['Guido']

type          HR  Temp
year visit            
2013 1      38.0  36.4
     2      36.0  36.6
2014 1      37.0  36.5
     2      37.0  38.5

In [33]:
# INDEXING AND SLICING A MULTIINDEX
# --- Multiply Indexed Series ---
pop

state       year
California  2000    33871648
            2010    37253956
New York    2000    18976457
            2010    19378102
Texas       2000    20851820
            2010    25145561
dtype: int64

In [30]:
pop['California', 2000]

33871648

In [31]:
pop['California']

year
2000    33871648
2010    37253956
dtype: int64

In [32]:
pop[pop > 22000000]

state       year
California  2000    33871648
            2010    37253956
Texas       2010    25145561
dtype: int64

In [34]:
# --- Multiply indexed DataFrames ---
health_data

subject      Bob       Guido         Sue      
type          HR  Temp    HR  Temp    HR  Temp
year visit                                    
2013 1      29.0  40.2  38.0  36.4  36.0  38.4
     2      31.0  36.9  36.0  36.6  20.0  37.3
2014 1      32.0  35.6  37.0  36.5  44.0  36.3
     2      50.0  37.2  37.0  38.5  37.0  37.5

In [35]:
 health_data['Guido', 'HR']

year  visit
2013  1        38.0
      2        36.0
2014  1        37.0
      2        37.0
Name: (Guido, HR), dtype: float64

In [36]:
health_data.iloc[:2, :2]

subject      Bob      
type          HR  Temp
year visit            
2013 1      29.0  40.2
     2      31.0  36.9

In [37]:
idx = pd.IndexSlice
health_data.loc[idx[:, 1], idx[:, 'HR']]

,subject,Bob,Guido,Sue
,type,HR,HR,HR
year,visit,,,
2013,1,29.0,38.0,36.0
2014,1,32.0,37.0,44.0


In [38]:
# REARRANGING MULTI INDICES
# --- Sorted and unsorted indices
index = pd.MultiIndex.from_product([['a', 'c', 'b'], [1, 2]])
data = pd.Series(np.random.rand(6), index=index)
data.index.names = ['char', 'int']
data

char  int
a     1      0.625887
      2      0.284660
c     1      0.221592
      2      0.820854
b     1      0.279582
      2      0.998807
dtype: float64

In [39]:
# If we try to take a partial slice of this index, it will result in an error:
try:
 data['a':'b']
except KeyError as e:
 print(type(e))
 print(e)


<class 'pandas.errors.UnsortedIndexError'>
'Key length (1) was greater than MultiIndex lexsort depth (0)'


In [40]:
data = data.sort_index()
data

char  int
a     1      0.625887
      2      0.284660
b     1      0.279582
      2      0.998807
c     1      0.221592
      2      0.820854
dtype: float64

In [41]:
data['a':'b']

char  int
a     1      0.625887
      2      0.284660
b     1      0.279582
      2      0.998807
dtype: float64

In [42]:
# --- Stacking and unstacking indices ---
pop.unstack(level=0)

state,California,New York,Texas
year,,,
2000,33871648,18976457,20851820
2010,37253956,19378102,25145561


In [43]:
pop.unstack(level=1)

year,2000,2010
state,,
California,33871648,37253956
New York,18976457,19378102
Texas,20851820,25145561


In [44]:
pop.unstack().stack()


state       year
California  2000    33871648
            2010    37253956
New York    2000    18976457
            2010    19378102
Texas       2000    20851820
            2010    25145561
dtype: int64

In [46]:
# --- Index setting and resetting ---
pop_flat = pop.reset_index(name='population')
pop_flat

,state,year,population
0,California,2000,33871648
1,California,2010,37253956
2,New York,2000,18976457
3,New York,2010,19378102
4,Texas,2000,20851820
5,Texas,2010,25145561


In [47]:
pop_flat.set_index(['state', 'year'])

population
state      year            
California 2000    33871648
           2010    37253956
New York   2000    18976457
           2010    19378102
Texas      2000    20851820
           2010    25145561

In [48]:
# DATA AGGREGATIONS ON MULTI-INDICES
health_data

subject      Bob       Guido         Sue      
type          HR  Temp    HR  Temp    HR  Temp
year visit                                    
2013 1      29.0  40.2  38.0  36.4  36.0  38.4
     2      31.0  36.9  36.0  36.6  20.0  37.3
2014 1      32.0  35.6  37.0  36.5  44.0  36.3
     2      50.0  37.2  37.0  38.5  37.0  37.5

In [49]:
data_mean = health_data.mean(level='year')
data_mean

subject   Bob        Guido         Sue       
type       HR   Temp    HR  Temp    HR   Temp
year                                         
2013     30.0  38.55  37.0  36.5  28.0  37.85
2014     41.0  36.40  37.0  37.5  40.5  36.90

In [50]:
data_mean.mean(axis=1, level='type')

type,HR,Temp
year,,
2013,31.666667,37.633333
2014,39.500000,36.933333
